**IMPORTS:**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as tsfm
import torchvision.utils

from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from torch import optim

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
import PIL.ImageOps


**Downloading data:**

In [ ]:
!wget https://github.com/maticvl/dataHacker/raw/master/DATA/at%26t.zip
!rm -rf data
!unzip "at&t.zip" -d .

--2022-07-16 06:17:35--  https://github.com/maticvl/dataHacker/raw/master/DATA/at%26t.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/maticvl/dataHacker/master/DATA/at%26t.zip [following]
--2022-07-16 06:17:35--  https://raw.githubusercontent.com/maticvl/dataHacker/master/DATA/at%26t.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3797060 (3.6M) [application/zip]
Saving to: ‘at&t.zip’

at&t.zip            100%[===================>]   3.62M  --.-KB/s    in 0.07s   

2022-07-16 06:17:36 (52.0 MB/s) - ‘at&t.zip’ saved [3797060/3797060]

Archive:  at&t.zip
   creating: ./data/
   creating: .

**Custom Dataset Class:**

Accepts 2 image inputs and 1 label

In [ ]:
class SNdataset(Dataset):

  def __init__(self, imageFolderDataset, transform=None):
    self.imageFolderDataset=imageFolderDataset
    self.transform=transform

  def __getitem__(self, index):
    img_tuple0=random.choice(self.imageFolderDataset.imgs) #get imgs randomly

    #to get 50% of imgs into the same class

    same_class=random.randint(0,1)

    if same_class:
      while True:

        img_tuple1=random.choice(self.imageFolderDataset.imgs) #look till same class img is found
        if img_tuple0[1] == img_tuple1[1]:
          break

    else:
      while True:

        img_tuple1=random.choice(self.imageFolderDataset.imgs) #else look till diff img class is found
        if img_tuple0[1] != img_tuple1[1]:
          break

    img0=Image.open(img_tuple0[0])
    img1=Image.open(img_tuple1[0])

    #greyscale

    img0=img0.convert("L")
    img1=img1.convert("L")

    if self.transform is not None:

      img0=self.transform(img0)
      img1=self.transform(img1)

    return img0, img1, torch.from_numpy(np.arr([int(img_tuple1[1] != img_tuple0[1])], dtype=np.float32))  
  def __len__(self):
    return len(self.imageFolderDataset.imgs)

**Loading Dataset:**

In [ ]:
path=datasets.ImageFolder(root="/content/data/faces/training")

In [ ]:
transformed=tsfm.Compose([
                             tsfm.Resize((100,100)),
                             tsfm.ToTensor()
                         ])

#intialize:
input_dataset=SNdataset(imageFolderDataset=path,transform=transformed)